In [2]:
from argparse import Namespace
import mlflow
import torch

import flash
from flash.text import TextClassificationData,TextClassifier
from flash.core.data.utils import download_data

In [3]:
args = Namespace(
    experiment_name = "mlflow1",
)

In [4]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment(args.experiment_name)

<Experiment: artifact_location='mlflow-artifacts:/271343097307036533', creation_time=1679553284714, experiment_id='271343097307036533', last_update_time=1679553284714, lifecycle_stage='active', name='mlflow1', tags={}>

In [5]:
experiment = mlflow.get_experiment_by_name(args.experiment_name)
experiment.experiment_id

'271343097307036533'

In [6]:
mlflow.pytorch.autolog()

2023/03/23 08:20:44 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of pytorch. If you encounter errors during autologging, try upgrading / downgrading pytorch to a supported version, or try upgrading MLflow.


In [7]:
download_data("https://pl-flash-data.s3.amazonaws.com/imdb.zip",
"../data/")


In [8]:
datamodule = TextClassificationData.from_csv(
    input_field="review",
    target_fields="sentiment",
    train_file="../data/imdb/train.csv",
    val_file="../data/imdb/valid.csv",
    test_file="../data/imdb/test.csv",
    batch_size=64,
)

Map:   0%|          | 0/22500 [00:00<?, ? examples/s]

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

/opt/conda/envs/test1/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3460: FutureWarning: Please pass an instantiated object of the `InputTransform` class. Passing the Class and keyword arguments separately has been deprecated since v0.8.0 and will be removed in v0.9.0.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [9]:
model = TextClassifier(backbone="prajjwal1/bert-tiny",
                       num_classes=datamodule.num_classes)

Using 'prajjwal1/bert-tiny' provided by Hugging Face/transformers (https://github.com/huggingface/transformers).
Some weights of the model checkpoint at prajjwal1/bert-tiny were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification

In [10]:
trainer = flash.Trainer(max_epochs=1,gpus=torch.cuda.device_count())

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [11]:
with mlflow.start_run(experiment_id=experiment.experiment_id,
                      run_name="mlflow_run_1"):
    trainer.finetune(model,datamodule,strategy="freeze")

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type               | Params
-----------------------------------------------------
0 | train_metrics | ModuleDict         | 0     
1 | val_metrics   | ModuleDict         | 0     
2 | test_metrics  | ModuleDict         | 0     
3 | adapter       | HuggingFaceAdapter | 4.4 M 
-----------------------------------------------------
258       Trainable params
4.4 M     Non-trainable params
4.4 M     Total params
17.545    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.
2023/03/23 08:21:22 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /var/tmp/tmpahct07is/model/data, flavor: pytorch), fall back to return ['torch==2.0.0', 'cloudpickle==2.2.1']. Set logging level to DEBUG to see the full traceback.
